In [1]:
import os

In [11]:
%pwd

'c:\\Users\\user\\Desktop\\Insurance E2E ML\\Insurance-Premium-End-to-End-'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\user\\Desktop\\Insurance E2E ML\\Insurance-Premium-End-to-End-'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    learning_rate: float
    target_column: str

In [6]:
from src.inproject.constants import *
from src.inproject.utils.common import read_yaml, create_directories

In [7]:
# for src configuration 
class ConfigurationManager:
    def __init__(self, 
                 config_pathway=CONFIG_FILE_PATH, 
                 param_pathway=PARAM_FILE_PATH, 
                 schema_pathway=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_pathway)
        self.param = read_yaml(param_pathway)
        self.schema = read_yaml(schema_pathway)

        create_directories([self.config.artifacts_root])#


    def get_model_trainer_config(self):
        config = self.config.model_trainer
        schema = self.schema.TARGET_COLUMN
        params = self.param.LGBMRegressor

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            n_estimators=params.n_estimators,
            learning_rate=params.learning_rate,
            target_column=schema.name
        )

        return model_trainer_config

In [8]:
import os
import pandas as pd
from src.inproject.logging import logger_re
# from sklearn.linear_model import ElasticNet
from lightgbm import LGBMRegressor
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lgbm = LGBMRegressor(n_estimators=self.config.n_estimators, learning_rate=self.config.learning_rate)
        lgbm.fit(train_x, train_y)

        joblib.dump(lgbm, os.path.join(self.config.root_dir, self.config.model_name))

In [10]:
try:
    config=ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()

except Exception as e:
    raise e

[2025-01-02 23:44:01,706: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-02 23:44:01,721: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-02 23:44:01,736: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-02 23:44:01,738: INFO: common: created directory at: artifacts]
[2025-01-02 23:44:01,740: INFO: common: created directory at: artifacts/model_trainer]
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022318 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1140
[LightGBM] [Info] Number of data points in the train set: 428133, number of used features: 19
[LightGBM] [Info] Start training from score -0.000000
